In [ ]:
import pandas as pd
import os
import json
import numpy as np
from itertools import groupby
import matplotlib.pyplot as plt
from scipy import stats,signal
import matplotlib as mpl
from sklearn.linear_model import LogisticRegression
import random
import re
import csv
from IPython.display import HTML, display, Image
import tabulate
import math as m
import warnings
warnings.filterwarnings('ignore')
import statsmodels.api as sm
import statsmodels.formula.api as smf

mpl.rcParams['lines.linewidth'] = 2
mpl.rcParams['axes.titlesize'] = 20
mpl.rcParams['axes.labelsize'] = 18
mpl.rcParams['lines.markersize'] = 10
mpl.rcParams['xtick.labelsize'] = 20
mpl.rcParams['ytick.labelsize'] = 20
mpl.rcParams['axes.linewidth'] = 1
#mpl.rcParams['xtick.major.size'] = 20
mpl.rcParams['xtick.major.width'] = 1
#mpl.rcParams['xtick.minor.size'] = 10
mpl.rcParams['xtick.minor.width'] = 1
mpl.rcParams['ytick.major.width'] = 1
mpl.rcParams['ytick.minor.width'] = 1
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False

fday = [1,2,3,4,5,6,7,8,9,10]
fsession = [1,2]
excluded = [(3062,1),(3062,2),(3062,4)]

In [ ]:
current_path = os.path.abspath(os.getcwd())
parent_path = os.path.abspath(os.path.join(current_path, os.pardir))
grand_parent_path = os.path.abspath(os.path.join(parent_path, os.pardir))
main_path = os.path.abspath(os.path.join(grand_parent_path, os.pardir))

path_results = main_path+'/results/dots/'
path_data = main_path+'/data/'

In [ ]:
def completeWithZero(domain, xlista, ylista):
    if len(domain)==len(xlista):
        return ylista
    elif len(domain)<len(xlista):
        print(domain,xlista)
        return 'Error'
    else:
        set1 = set(domain)
        set2 = set(xlista)
        # missed values
        missing = list(sorted(set1 - set2))
        # index of missed values
        index = []
        [index.append(domain.index(elem)) for elem in missing]
        # insert zero in ylista
        [ylista.insert(elem,0) for elem in index]
        return ylista

In [ ]:
# list of participants 
participants = [3051+i for i in range(28)]

In [ ]:
accQ,accF,accS,DaccNO,SaccNO,accNO,globalDaccNO,globalSaccNO = [],[],[],[],[],[],[],[] # list of arrays of acc per RT quartile for every subject and session
for day in fday:
    for session in fsession:
        sessionid = 2*day-2+session
        
        path = path_data+'jatos_dots_data/tanda1/day'+str(day)+'/session'+str(session)+'/'
        data_files = [f for f in os.listdir(path) if f.endswith('_day'+str(day)+'_session'+str(session))]
        
        # sort files
        subj_data = [int(re.search('%s(.*)%s' % ('', '_day'), f).group(1)) for f in data_files]
        sorted_subj_data = sorted(subj_data)
        index_subj_data = [subj_data.index(elem) for elem in sorted_subj_data]
        sorted_data_files = [data_files[i] for i in index_subj_data]

        Ddf,Sdf = {},{}
        for name in sorted_subj_data: 
            Ddf[name] = pd.DataFrame()
            Sdf[name] = pd.DataFrame()
                       
        ind = -1
        for ses in sorted_data_files:
            ind += 1
            data = [] 
            for line in open(path+ses, 'r'):
                if line.strip():
                    data.append(json.loads(line))

            if day==1 and session==1:
                if len(data)==5:
                    deterministic = data[3]
                    stochastic = data[4]
                else:
                    deterministic = data[len(data)-2]
                    stochastic = data[len(data)-1]
            else:
                if len(data)==4:
                    deterministic = data[2]
                    stochastic = data[3]
                elif len(data)>4:
                    deterministic = data[len(data)-2]
                    stochastic = data[len(data)-1]
                    print('participant '+ str(sorted_subj_data[ind])+' has repeated practice '+str(len(data)-3))
                    pract = []
                    for j in range(len(data)):
                        if len(data[j])==10:
                            pract.append(data[j])
                else: 
                    for dd in range(len(data)):
                        print(len(data[dd]))
                    deterministic = data[2]
                    print('participant '+ str(sorted_subj_data[ind])+' missed stochastic')       
            Ddf[sorted_subj_data[ind]] = pd.DataFrame.from_dict(deterministic)
            Sdf[sorted_subj_data[ind]] = pd.DataFrame.from_dict(stochastic)

        for part in sorted_subj_data:
            if (part,sessionid) not in excluded:
                ## deterministic dataframe
                # discrimination RT
                discrimination_RT = np.array(list(Ddf[part].discrimination_t_keydown))-np.array(list(Ddf[part].t_offset))
                Ddf[part]["discrimination_RT"] = discrimination_RT 
                Ddf[part]['signal'] = np.abs(Ddf[part]['dots_num_left']-Ddf[part]['dots_num_right'])
                Dsignal = np.array(np.abs(Ddf[part]['dots_num_left']-Ddf[part]['dots_num_right']))/100
                Dorientation = Ddf[part]['orientation']
                signedDsignal = []
                ind = -1
                for elem in Dorientation:
                    ind += 1
                    if elem:
                        signedDsignal.append(Dsignal[ind])
                    else: 
                        signedDsignal.append(-Dsignal[ind])  
                # signed signal (neg signal to the left and pos signal tothe right)
                Ddf[part]['signed_signal'] = signedDsignal
                Dcorrect_list = list(Ddf[part]['discrimination_is_correct'])
                Doptout_list = list(Ddf[part]["optout"])
                Dbool_correct_list = [elem==1 for elem in Dcorrect_list]
                Dbool_orientation_list = [elem==1 for elem in Dorientation]
                Dxor = np.logical_xor(Dbool_correct_list,np.logical_not(Dbool_orientation_list))
                Dresp = []
                ind = -1
                for elem in Doptout_list:
                    ind += 1
                    if elem==False and Dxor[ind]==True:
                        Dresp.append(1)
                    elif elem==False and Dxor[ind]==False:
                        Dresp.append(0)
                    elif elem==True: 
                        Dresp.append(2)
                # 1 for rightward answers, 0 for left
                Ddf[part]['resp_is_R'] = [elem==1 for elem in Dresp]
                Ddf[part]['resp_is_R'] = Ddf[part]['resp_is_R'].astype(int)
                # 1 for rightward answers, 0 for left, and 2 for optout
                Ddf[part]['answer'] = Dresp

                ## stochastic dataframe
                # discrimination RT
                Sdiscrimination_RT = np.array(list(Sdf[part].discrimination_t_keydown))-np.array(list(Sdf[part].t_offset))
                Sdf[part]["discrimination_RT"] = Sdiscrimination_RT 
                Sdf[part]['signal'] = np.abs(Sdf[part]['dots_num_left']-Sdf[part]['dots_num_right'])
                Ssignal = np.array(np.abs(Sdf[part]['dots_num_left']-Sdf[part]['dots_num_right']))/100
                Sorientation = Sdf[part]['orientation']
                signedSsignal = []
                ind = -1
                for elem in Sorientation:
                    ind += 1
                    if elem:
                        signedSsignal.append(Ssignal[ind])
                    else: 
                        signedSsignal.append(-Ssignal[ind])  
                # signed signal (neg signal to the left and pos signal tothe right)
                Sdf[part]['signed_signal'] = signedSsignal
                Scorrect_list = list(Sdf[part]['discrimination_is_correct'])
                Soptout_list = list(Sdf[part]["optout"])
                Sbool_correct_list = [elem==1 for elem in Scorrect_list]
                Sbool_orientation_list = [elem==1 for elem in Sorientation]
                Sxor = np.logical_xor(Sbool_correct_list,np.logical_not(Sbool_orientation_list))
                Sresp = []
                ind = -1
                for elem in Soptout_list:
                    ind += 1
                    if elem==False and Sxor[ind]==True:
                        Sresp.append(1)
                    elif elem==False and Sxor[ind]==False:
                        Sresp.append(0)
                    elif elem==True: 
                        Sresp.append(2)
                # 1 for rightward answers, 0 for left
                Sdf[part]['resp_is_R'] = [elem==1 for elem in Sresp]
                Sdf[part]['resp_is_R'] = Sdf[part]['resp_is_R'].astype(int)
                # 1 for rightward answers, 0 for left, and 2 for optout
                Sdf[part]['answer'] = Sresp
            
        # deterministic non-optout & optout df
        Ddf_no,Ddf_oo = {},{}
        for name in sorted_subj_data: 
            Ddf_no[name] = pd.DataFrame()
            Ddf_oo[name] = pd.DataFrame()
        for part in sorted_subj_data:
            Ddf_no[part] = Ddf[part][(Ddf[part]['focus']==0)]
            Ddf_oo[part] = Ddf[part][(Ddf[part]['focus']==1)]
                
        # stochastic non-optout & optout df
        Sdf_no,Sdf_oo = {},{}
        for name in sorted_subj_data: 
            Sdf_no[name] = pd.DataFrame()
            Sdf_oo[name] = pd.DataFrame()
        for part in sorted_subj_data:
            Sdf_no[part] = Sdf[part][(Sdf[part]['focus']==0)]
            Sdf_oo[part] = Sdf[part][(Sdf[part]['focus']==1)]

        # non-optout df for each participant
        df_no = {}
        for name in sorted_subj_data: 
            df_no[name] = pd.DataFrame()
        for part in sorted_subj_data:
            
            if (part,sessionid) not in excluded:
            
                df_no[part] = pd.concat([Ddf_no[part],Sdf_no[part]])

                df_no[part] = df_no[part].astype({'discrimination_is_correct': 'float'})
                dfdiffNO = df_no[part].groupby(['difficulty'])[['discrimination_is_correct']].mean().reset_index()
                dfdiffNO = dfdiffNO.sort_values('difficulty')
                accNO.append(100*dfdiffNO['discrimination_is_correct'])
                
                Ddf_no[part] = Ddf_no[part].astype({'discrimination_is_correct': 'float'})
                dfDdiffNO = Ddf_no[part].groupby(['difficulty'])[['discrimination_is_correct']].mean().reset_index()
                dfDdiffNO = dfDdiffNO.sort_values('difficulty')
                # accuracy in deterministic NO trials
                DaccNO.append(100*dfDdiffNO['discrimination_is_correct'])
                globalDaccNO.append(100*Ddf_no[part]['discrimination_is_correct'].mean())
                
                Sdf_no[part] = Sdf_no[part].astype({'discrimination_is_correct': 'float'})
                dfSdiffNO = Sdf_no[part].groupby(['difficulty'])[['discrimination_is_correct']].mean().reset_index()
                dfSdiffNO = dfSdiffNO.sort_values('difficulty')
                # accuracy in stochastic NO trials
                SaccNO.append(100*dfSdiffNO['discrimination_is_correct'])
                globalSaccNO.append(100*Sdf_no[part]['discrimination_is_correct'].mean())

                # medianRT
                RT_list = list(df_no[part]['discrimination_t_keydown'])+list(Ddf_oo[part]['discrimination_t_keydown'])+\
                list(Sdf_oo[part]['discrimination_t_keydown'])
                # global median RT for this participant
                medianRT = np.nanmedian(RT_list)

                # normalized RT 
                df_no[part]['norm_NO_RT'] = df_no[part]['discrimination_t_keydown']*(1./medianRT)
                # sort rows by normalized NO RT
                df_no[part] = df_no[part].sort_values('norm_NO_RT') 
                # index for each element
                df_no[part]['idx'] = np.arange(df_no[part].shape[0])
                # cast index in interval 0 - 100
                df_no[part]['idx'] = df_no[part]['idx']/(df_no[part]['idx'].max()+1)*100 
                # convert discrimination_is_correct and signed_signal to float
                df_no[part] = df_no[part].astype({'discrimination_is_correct': 'float'})
                df_no[part] = df_no[part].astype({'signed_signal': 'float'})
                # columns to average
                cols2mean = ['discrimination_is_correct','discrimination_t_keydown','norm_NO_RT']

                # fast and slow dfs
                dfS, dfF = df_no[part][(mask:=df_no[part]['idx'] >= 50)], df_no[part][~mask]

                '''
                # mean of every column group group by signed_signal
                dfSsig = dfS.groupby(['signed_signal'])[cols2mean].mean().reset_index()
                dfSsig = dfSsig.sort_values('signed_signal') 
                dfFsig = dfF.groupby(['signed_signal'])[cols2mean].mean().reset_index()
                dfFsig = dfFsig.sort_values('signed_signal') 
                # mean accuracy per signed signal
                accSmean = list(100*dfSsig['discrimination_is_correct'])
                accFmean = list(100*dfFsig['discrimination_is_correct'])

                # every values for signed signal
                signed_signals = [-0.14, -0.1 , -0.06, -0.02,  0.02,  0.06,  0.1 ,  0.14]
                # unique signed signal
                Sunique = list(dfSsig['signed_signal'].unique())
                Funique = list(dfFsig['signed_signal'].unique())

                accSmean = completeWithZero(signed_signals, Sunique, accSmean)
                accFmean = completeWithZero(signed_signals, Funique, accFmean)
                '''
                # mean of every column group group by difficulty
                dfSsig = dfS.groupby(['difficulty'])[cols2mean].mean().reset_index()
                dfSsig = dfSsig.sort_values('difficulty') 
                dfFsig = dfF.groupby(['difficulty'])[cols2mean].mean().reset_index()
                dfFsig = dfFsig.sort_values('difficulty')             
                # mean accuracy per signed signal
                accSmean = list(100*dfSsig['discrimination_is_correct'])
                accFmean = list(100*dfFsig['discrimination_is_correct'])

                # every values for difficulty
                diff_values = [1,2,3,4]
                # unique difficulty
                Sunique = list(dfSsig['difficulty'].unique())
                Funique = list(dfFsig['difficulty'].unique())

                accSmean = completeWithZero(diff_values, Sunique, accSmean)
                accFmean = completeWithZero(diff_values, Funique, accFmean)

                accS.append(accSmean)
                accF.append(accFmean)

                # define quartiles
                df_no[part]['quartile'] = df_no[part]['idx'] // 25 
                # mean of every column group by RT quartile 
                dfQ = df_no[part].groupby(['quartile'])[cols2mean].mean().reset_index()
                # mean accuracy per quartile of RT
                accQ.append(np.array(100*dfQ['discrimination_is_correct']))


In [ ]:
for diff in np.arange(4):
    ttest = stats.ttest_rel(np.array(DaccNO)[:,diff],np.array(SaccNO)[:,diff])
    print(ttest)

In [ ]:
stats.ttest_rel(np.array(globalDaccNO),np.array(globalSaccNO))

In [ ]:
meanACC_DNO = np.array(DaccNO).mean(axis=0) 
semACC_DNO = np.array(DaccNO).std(axis=0)/np.sqrt(len(DaccNO))

In [ ]:
meanACC_SNO = np.array(SaccNO).mean(axis=0) 
semACC_SNO = np.array(SaccNO).std(axis=0)/np.sqrt(len(SaccNO))

In [ ]:
plt.plot(np.arange(4)+1,meanACC_DNO,color=[1,0,0.4])
plt.fill_between(np.arange(4)+1,np.array(meanACC_DNO)-np.array(semACC_DNO), 
                                 np.array(meanACC_DNO)+np.array(semACC_DNO), alpha=0.2, color = [1,0,0.4])
plt.plot(np.arange(4)+1,meanACC_SNO,color=[0.4,0,1])
plt.fill_between(np.arange(4)+1,np.array(meanACC_SNO)-np.array(semACC_SNO), 
                                 np.array(meanACC_SNO)+np.array(semACC_SNO), alpha=0.2, color = [0.4,0,1])
# plt.ylabel('accuracy (%)')
plt.xlabel('difficulty')
plt.legend(['DO','SO'],loc='best', shadow=True,fontsize=18)
plt.yticks([60,80],labels=[])
plt.ylim(45,99)
plt.tight_layout()
plt.savefig('acc_NO_D&S.png')
plt.show()

In [ ]:
meanACCS = np.array(accS).mean(axis=0) 
semACCS = np.array(accS).std(axis=0)/np.sqrt(len(accS))

In [ ]:
meanACCF = np.array(accF).mean(axis=0) 
semACCF = np.array(accF).std(axis=0)/np.sqrt(len(accS))

In [ ]:
plt.plot(np.arange(4)+1,meanACCF,color='gray')
plt.fill_between(np.arange(4)+1,np.array(meanACCF)-np.array(semACCF), 
                                 np.array(meanACCF)+np.array(semACCF), alpha=0.2, color = 'grey')
plt.plot(np.arange(4)+1,meanACCS,color='k')
plt.fill_between(np.arange(4)+1,np.array(meanACCS)-np.array(semACCS), 
                                 np.array(meanACCS)+np.array(semACCS), alpha=0.2, color = 'k')

# plt.ylabel('accuracy (%)')
plt.xlabel('difficulty')
plt.legend(['fast','slow'],loc='best', shadow=True,fontsize=18)
plt.ylim(45,99)
plt.yticks([60,80],labels=[])
plt.tight_layout()
plt.savefig('acc_fast&slow_diff.png')
plt.show()

In [ ]:
meanACCQ = np.array(accQ).mean(axis=0) 
semACCQ = np.array(accQ).std(axis=0)/np.sqrt(len(accQ))

In [ ]:
plt.plot([1,2,3,4],meanACCQ,color='k')
plt.fill_between([1,2,3,4],np.array(meanACCQ)-np.array(semACCQ), 
                                 np.array(meanACCQ)+np.array(semACCQ), alpha=0.2, color = 'k')
plt.ylabel('accuracy (%)')
plt.xlabel('RT')
plt.ylim(45,99)
plt.tight_layout()
plt.xticks([1,2,3,4],labels=['q1','q2','q3','q4'])
plt.savefig('acc_RTquartiles.png')
plt.show()

In [ ]:
dict_ = {'NOdetermistic':{'mean':list(meanACC_DNO),'sem':list(semACC_DNO)},\
         'NOstochastic':{'mean':list(meanACC_SNO),'sem':list(semACC_SNO)},\
        'accFast':{'mean':list(meanACCF),'sem':list(semACCF)},\
        'accSlow':{'mean':list(meanACCS),'sem':list(semACCS)},
        'accRTquartile':{'mean':list(meanACCQ),'sem':list(semACCQ)}}

In [ ]:
### DO NOT RUN AGAIN !!!

# write the result in file
filename=path_results+'confidence_extra_analysis.json'
# Serializing json  
json_object_ = json.dumps(dict_) 

# Writing to sample.json 
with open(filename, "w") as outfile: 
    outfile.write(json_object_) 

In [ ]:
list(df_no[part]['quartile'].value_counts())

In [ ]:
df_no[part]['quartile'].unique()

In [ ]:
dfQ.head()

In [ ]:
df_no[part].columns

In [ ]:
np.array(accQ)

In [ ]:
df_no[part]['discrimination_is_correct'].mean()

In [ ]:
len(df_no[3062])

In [ ]:
df_no[3062] = df_no[3062].sort_values('v2') # Ordenas por el valor
d['idx'] = np.arange(d.shape[0]) # le ponés un índice a cada elemento
d['idx'] = d['idx'] / (d['idx'].max() + 1) * 100 # llevas esos índices al intervalo 0 ... 100
d['quartile'] = d['idx'] // 25 #

In [ ]:
len(Ddf[3062])

In [ ]:
df_no[3062].columns

In [ ]:
n = 100_000
by = 18_000
df = pd.DataFrame(np.random.uniform(0, 1, (n, 3)))

In [ ]:
df

In [ ]:
out = df.groupby(df.index // by).apply(pd.DataFrame.quantile, .75)

In [ ]:
df.groupby(df.index // by).head()

In [ ]:
out